In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [3]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

data=datasets.ImageFolder('C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\TryCNN\\animals\\animals',transform=transform)

In [4]:
print(data[0])

(tensor([[[0.7412, 0.7373, 0.7412,  ..., 0.8039, 0.7882, 0.7765],
         [0.7412, 0.7373, 0.7373,  ..., 0.8118, 0.7882, 0.7843],
         [0.7451, 0.7451, 0.7373,  ..., 0.8118, 0.7882, 0.7961],
         ...,
         [0.7137, 0.6118, 0.6392,  ..., 0.7765, 0.6941, 0.7451],
         [0.6275, 0.5216, 0.6902,  ..., 0.7843, 0.6980, 0.7333],
         [0.5176, 0.5569, 0.6510,  ..., 0.7294, 0.6627, 0.6706]],

        [[0.7098, 0.7059, 0.7098,  ..., 0.7412, 0.7176, 0.7059],
         [0.7098, 0.7059, 0.7098,  ..., 0.7569, 0.7294, 0.7216],
         [0.7137, 0.7137, 0.7137,  ..., 0.7686, 0.7490, 0.7490],
         ...,
         [0.6706, 0.5412, 0.5922,  ..., 0.7686, 0.7059, 0.7529],
         [0.6039, 0.4745, 0.6667,  ..., 0.7882, 0.7333, 0.7373],
         [0.4941, 0.5412, 0.6353,  ..., 0.7490, 0.7059, 0.6667]],

        [[0.6275, 0.6235, 0.6235,  ..., 0.6471, 0.6275, 0.6118],
         [0.6275, 0.6235, 0.6157,  ..., 0.6745, 0.6510, 0.6510],
         [0.6314, 0.6314, 0.6157,  ..., 0.6980, 0.6784, 0

In [5]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_size

240

In [6]:
train_data,test_data=torch.utils.data.random_split(data,[train_size,test_size])

In [7]:
train_loader = DataLoader(train_data, batch_size=5, shuffle=True)
test_loader = DataLoader(test_data, batch_size=5, shuffle=False)

In [8]:
con1=nn.Conv2d(3,32,3)
con2=nn.Conv2d(32,64,3)
con3=nn.Conv2d(64,128,3)

fc1=nn.Linear(26*26*128, 512)
fc2=nn.Linear(512, 256)
fc3=nn.Linear(256, 90)

pool = nn.MaxPool2d(2)

device = torch.device("cuda")

con1.to(device)
con2.to(device)
con3.to(device)

fc1.to(device)
fc2.to(device)
fc3.to(device)

Linear(in_features=256, out_features=90, bias=True)

In [9]:
def forward(x):
    x = pool(F.relu(con1(x)))
    x = pool(F.relu(con2(x)))
    x = pool(F.relu(con3(x)))

    x = x.view(x.size(0), -1)

    x = F.relu(fc1(x))
    x = F.relu(fc2(x))
    x = fc3(x)
    return x
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(con1.parameters()) + list(con2.parameters()) + list(con3.parameters()) + 
                       list(fc1.parameters()) + list(fc2.parameters()) + list(fc3.parameters()))


In [10]:
for epoch in range(100):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = forward(inputs)
        
        if labels.dim() > 1:
            labels = labels.squeeze(1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss}, Accuracy: {epoch_accuracy}%")

Epoch 1, Loss: 2.0232409362991652, Accuracy: 25.0%
Epoch 2, Loss: 1.5756691383818786, Accuracy: 32.083333333333336%
Epoch 3, Loss: 1.3113444931805134, Accuracy: 45.0%
Epoch 4, Loss: 1.312202338129282, Accuracy: 50.416666666666664%
Epoch 5, Loss: 1.221193299939235, Accuracy: 50.0%
Epoch 6, Loss: 1.057222173238794, Accuracy: 60.416666666666664%
Epoch 7, Loss: 0.7910472620278597, Accuracy: 74.58333333333333%
Epoch 8, Loss: 0.6657809124638637, Accuracy: 78.33333333333333%
Epoch 9, Loss: 0.7357299323193729, Accuracy: 78.75%
Epoch 10, Loss: 0.4309390818234533, Accuracy: 85.83333333333333%
Epoch 11, Loss: 0.349677730545712, Accuracy: 88.33333333333333%
Epoch 12, Loss: 0.38477722912405926, Accuracy: 87.5%
Epoch 13, Loss: 0.18621399237114625, Accuracy: 96.66666666666667%
Epoch 14, Loss: 0.20018116187808724, Accuracy: 93.75%
Epoch 15, Loss: 0.14055039779236722, Accuracy: 93.75%
Epoch 16, Loss: 0.13222124631264856, Accuracy: 96.25%
Epoch 17, Loss: 0.06621219869036092, Accuracy: 97.5%
Epoch 18, Lo

In [11]:
torch.save({
    'con1_state_dict': con1.state_dict(),
    'con2_state_dict': con2.state_dict(),
    'con3_state_dict': con3.state_dict(),
    'fc1_state_dict': fc1.state_dict(),
    'fc2_state_dict': fc2.state_dict(),
    'fc3_state_dict': fc3.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'run.pth')